In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup, element
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from selenium.common.exceptions import NoSuchElementException

Webscraping:

Create a program that can extract information from housing aggregator websites like gratka.pl or otodom.pl. The program should be able to extract all the relevant data from a search query, such as location, minimum and maximum price, minimum and maximum area, and other important features. It should also include options for private or public ads. The extracted data should include the link to the record, price, number of rooms, area, description, other features, and position on a map. You can extract the position using Google Maps iframes.

 

Once you have extracted the data, use it to gain insights about a smaller town. The program should allow users to input the town they are interested in and then extract all the relevant information about available housing in that area. This will provide useful information for those who are looking to move to a new town or city, or for anyone who is interested in the housing market in a particular area.

In [ ]:
### There is an issue with the code there should be omit allers included

## Gratka.pl

In [87]:
def gratka_df(city:str, num_of_pages:int):
    ### 1 Step
    # Opening Google
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    # Direct to predefine website
    driver.get('https://gratka.pl/')

    ### 1.5 Step
    # Accespting cookies
    time.sleep(2)
    cookie = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[2]/div/div[6]/button[2]')
    cookie.click()
    time.sleep(1)

    ### 2 Step
    ## Navigate to real-estate
    real_estates = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[1]/div[2]/ul/li[1]/a')
    real_estates.click()
    # All categories
    all_estates = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[1]/div[2]/ul/li[2]/ul/li[8]/a')
    all_estates.click()
    # location
    loc_bar = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[2]/div/div/div[1]/input')
    loc_bar.send_keys(city) #variable
    time.sleep(2)
    loc_bar.send_keys(Keys.ENTER)

    ### 2.5 Step
    ## Search
    search = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[9]/div')
    search.click()
    time.sleep(2)
    search = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div/div[9]/div')
    search.click()
    time.sleep(2)

    ### 4 Iterate through houses
    soup = BeautifulSoup(driver.page_source)
    houses = list()
    df_gratka = pd.DataFrame()

    # retriving info about number of pages
    max_pages = int(soup.find('div', {'class':'pagination container'}).text.replace('\n','').split('z')[-1])

    if num_of_pages < max_pages:
        x = num_of_pages
    else:
        x = max_pages

    for i in range(0,x):
        # dealing with alerts popouts
        try:
            time.sleep(2)
            driver.find_element(By.XPATH, '/html/body/div[4]/div/div[2]/div[1]/div[1]/div/button').click() 
        except NoSuchElementException:
            pass

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        properties = soup.find_all('div', {'class': 'listing__teaserWrapper'})
        for proper in properties:
            houses.append({
                'name': proper.find('h2',{'class':'teaserUnified__title'}).text.split('\n')[1],
                'city': proper.find('span', {'class': 'teaserUnified__location'}).text.split(',')[0].replace(' ','').replace('\n',''),
                'price' : proper.find('p', {'class': 'teaserUnified__price'}).text.split('\n')[1],
                'area' : proper.find('li', {'class': 'teaserUnified__listItem'}).text,
                'price_per_m2' : (proper.find('span', {'class': 'teaserUnified__additionalPrice'})).text.split('\n')[1],
                'link': proper.find('a', {'class': 'teaserLink'}).get('href'),
                #'District':  proper.find('span', {'class': 'teaserUnified__location'}).text.split(',')[1].replace(' ',''),
                #'Street': proper.find('h2', {'class': 'teaserUnified__title'}).text.split(',')[-1].replace('\n',''),
                })
        df = pd.DataFrame(houses)

        additions = list()
        for x in range(len(df['link'])):
            url = df['link'][x]
            res = requests.get(url)
            soup = BeautifulSoup(res.text, 'html.parser')
            try:
                forma_wlasnosci = soup.find_all('b', {'class': 'parameters__value'})[4].text
            except:
                forma_wlasnosci = None             
            try:
                stan_wykonczenia = soup.find('b', {'class': 'parameters__value'}).find_all('li')[3].text
            except:
                stan_wykonczenia = None               
            try:
                pietro = soup.find_all('b', {'class': 'parameters__value'})[3].text
            except:
                pietro = None      
            try:
                parking = soup.find_all('b', {'class': 'parameters__value'})[7].text
            except:
                parking = None
            try:
                rooms = soup.find_all('b', {'class': 'parameters__value'})[2].text
            except:
                rooms = None
                        
            additions.append({
                'forma_wlasnosci': forma_wlasnosci,
                'stan_wykonczenia': stan_wykonczenia,
                'pietro': pietro,
                'parking': parking,
                'rooms': rooms,
                'balkon':None,	
                'czynsz':None,	
                'ogrzewanie':None
                })
            
        df = df.join(pd.DataFrame(additions))
        df_gratka = df_gratka.append(df)

        if i == 0:   
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[43]/a[6]').click()
        elif i > 0 and i+1 < x:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[43]/a[7]').click()
        else:
            break
                
    driver.quit()
    df_gratka_v2 = df_gratka.drop_duplicates()
    return df_gratka_v2

In [73]:
df_gratka_v2 = df_gratka.drop_duplicates()

In [77]:
df_gratka_v2.forma_wlasnosci[2].isdigit()

True

In [83]:
if df_gratka_v2.parking[0] is None:
    print('o')

o


In [86]:
for row in range(0,len(df_gratka_v2)):
    if df_gratka_v2.forma_wlasnosci[row].isdigit() is True:
        df_gratka_v2.forma_wlasnosci[row] = None
    else:
        pass
    
    if df_gratka_v2.parking[0] is None:
        pass
    elif df_gratka_v2.parking[row].isdigit() is True:
        df_gratka_v2.parking[row] = None
    else:
        pass

    if df_gratka_v2.parking[0] is None:
        pass
    elif df_gratka_v2.forma_wlasnosci[row].isdigit() is False:
        df_gratka_v2.forma_wlasnosci[row] = None
    else:
        pass

AttributeError: 'NoneType' object has no attribute 'isdigit'

In [80]:
isinstance(df_gratka_v2.forma_wlasnosci[0], str)

True

In [85]:
df_gratka_v2

,name,city,price,area,price_per_m2,link,forma_wlasnosci,stan_wykonczenia,pietro,parking,rooms,balkon,czynsz,ogrzewanie
0,"Mieszkanie Kraków Płaszów, ul. Mały Płaszów",Kraków,498 000 zł,"56,60 m2",8 799 zł/m2,https://gratka.pl/nieruchomosci/mieszkanie-kra...,własność,None,4,None,2,None,None,None
1,"Mieszkanie Kraków Krowodrza, ul. Łokietka",Kraków,800 000 zł,61 m2,13 115 zł/m2,https://gratka.pl/nieruchomosci/mieszkanie-kra...,własność,None,parter,None,2,None,None,None
2,"Nowe mieszkanie Kraków Borek Fałęcki, ul...",Kraków,563 112 zł,"50,29 m2",11 197 zł/m2,https://gratka.pl/nieruchomosci/nowe-mieszkani...,None,None,5,None,2,None,None,None
3,3-pokojowe os. Bohaterów Września - od ...,Kraków,529 000 zł,"55,41 m2",9 547 zł/m2,https://gratka.pl/nieruchomosci/3-pokojowe-os-...,None,None,3,1980,"55,41 m2",None,None,None
4,"Nowe mieszkanie Kraków Borek Fałęcki, ul...",Kraków,548 654 zł,"50,29 m2",10 910 zł/m2,https://gratka.pl/nieruchomosci/nowe-mieszkani...,None,None,1,None,2,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Mieszkanie Kraków Krowodrza, ul. Friedleina",Kraków,525 000 zł,50 m2,10 500 zł/m2,https://gratka.pl/nieruchomosci/mieszkanie-kra...,do remontu,None,parter,4,2,None,None,None
92,"Nowe mieszkanie Kraków Prądnik Biały, ul...",Kraków,678 776 zł,"57,04 m2",11 900 zł/m2,https://gratka.pl/nieruchomosci/nowe-mieszkani...,własność,None,6,7,3,None,None,None
93,"Mieszkanie Kraków Podgórze, ul. Mariana ...",Kraków,640 000 zł,61 m2,10 492 zł/m2,https://gratka.pl/nieruchomosci/mieszkanie-kra...,None,None,1,None,4,None,None,None
94,"Nowe mieszkanie Kraków Płaszów, ul. Myśl...",Kraków,585 000 zł,"55,60 m2",10 522 zł/m2,https://gratka.pl/nieruchomosci/nowe-mieszkani...,None,None,1,gratka-WE146408,3,None,None,None
